<a href="https://colab.research.google.com/github/YolandaMDavis/wildtrack-iqa/blob/task-amenability/cropping_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ensure to upload the provided "Faster R-CNN resnet50 (best single).zip" file along with a quality 5, quality 1 images and a non footprint image before running.**

In [2]:
#Only needed to copy data to local drive can be skipped if zip file is already available in working folder
import shutil

# mount google drive
from google.colab import drive
drive.mount('/content/drive') # for google colab. adjust accordingly
PARENT_DIR = '/content/drive/MyDrive/Wildtrack Group/IQA' 

# copy and extract tar file
shutil.copy(PARENT_DIR + '/data/WildTrack_Raw.zip', 'WildTrack_Raw.zip')

Mounted at /content/drive


'WildTrack_Raw.zip'

In [3]:
from zipfile import ZipFile

# extract zip file
with ZipFile('WildTrack_Raw.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [4]:
import torch
import torchvision
from PIL import Image
import torchvision.transforms as T
import matplotlib.pyplot as plt
import cv2
import json
import numpy as np
import albumentations as A

shutil.copy(PARENT_DIR + '/task-amenability/object/model.pt', 'object_model.pt')
shutil.copy(PARENT_DIR + '/task-amenability/object/class_mapping.json', 'class_mapping.json')
shutil.copy(PARENT_DIR + '/task-amenability/object/transforms.json', 'transforms.json')

with open('class_mapping.json') as data:
    mappings = json.load(data)

class_mapping = {item['model_idx']: item['class_name'] for item in mappings}
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.jit.load('object_model.pt').to(device)
transforms = A.load('transforms.json')

In [18]:
import os

def crop_image(image_location):

  image = Image.open(image_location)
  bands = image.getbands()
  image = np.array(image)

  if len(bands) == 1:
    image = np.expand_dims(image, axis=2)  
    
  image = transforms(image=image)['image']
  x = torch.from_numpy(image).to(device)
  
  with torch.no_grad():
    x = x.permute(2, 0, 1).float()
    y = model(x)

    # remove overlapped boxes
    no_overlap = torchvision.ops.nms(y['pred_boxes'], y['scores'], 0.5)
    y['scores'] = y['scores'][no_overlap]
    y['pred_boxes'] = y['pred_boxes'][no_overlap]
    y['pred_classes'] = y['pred_classes'][no_overlap]

    # keep only predictions with scores greater than .5
    to_keep = (y['scores'] > 0.5).nonzero(as_tuple=True)[0]
    y['pred_boxes'] = y['pred_boxes'][to_keep]
    y['pred_classes'] = y['pred_classes'][to_keep] 

    if len(y['pred_boxes'].tolist()) > 0:
      pred_box = y['pred_boxes'].tolist()[0]
      x = int(pred_box[0])
      y = int(pred_box[1])
      w = int(pred_box[2])
      h = int(pred_box[3])
      cropped_image = image[y:y+h, x:x+w]

      if len(bands) == 1:
        cropped_image = np.squeeze(cropped_image, axis=2)

      return Image.fromarray(cropped_image)

    else:
      return None

def generate_data_sets(rootdir='CROPPED'):

    image_reference_list = []
    subdirectories = list(os.walk('RAW', topdown=False))[:-1]
    
    for subdir in subdirectories:
        image_location = subdir[0]
        images = subdir[2]
        species_rating = image_location.rsplit('/', 1)[-1].replace('_', ' ')
        subjective_score = int(species_rating.rsplit(' ', 1)[-1])
        species_class = species_rating.rsplit(' ', 1)[:-1][0]

        if len(species_class.rsplit(' ', 1)) > 1:
            species = species_class.rsplit(' ')[0]
            animal_class = ' '.join(species_class.rsplit(' ')[1:])
        else:
            animal_class = 'Unknown'
            species = species_class

        os.makedirs(rootdir + '/' + image_location)    

        for image in images:
            image_path = image_location + '/' + image
            cropped_img = crop_image(image_location + '/' + image)
            if cropped_img is not None:
              cropped_img_path = rootdir + '/' + os.path.splitext(image_path)[0] + '_.jpg'
              print(cropped_img_path)
              cropped_img.save(cropped_img_path)
            

In [ ]:
rootdir = 'CROPPED'
generate_data_sets(rootdir)

CROPPED/RAW/Tiger_Amur_5/19e31b62d3de11ea91590242ac1c0002_.jpg
CROPPED/RAW/Tiger_Amur_5/14789bacd3de11ea91590242ac1c0002_.jpg
CROPPED/RAW/Tiger_Amur_5/1e0b05e2d3de11ea91590242ac1c0002_.jpg
CROPPED/RAW/Tiger_Amur_5/18a3eaa6d3de11ea91590242ac1c0002_.jpg
CROPPED/RAW/Tiger_Amur_5/1773848ed3de11ea91590242ac1c0002_.jpg
CROPPED/RAW/Tiger_Amur_5/185ffde6d3de11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/ade563d4d3dd11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/b2b56b02d3dd11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/bbc3fcd6d3dd11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/f2cb6e06c48411eb9d2b00155db4bfaf_.jpg
CROPPED/RAW/Puma_4/da681792c48411eb9d2b00155db4bfaf_.jpg
CROPPED/RAW/Puma_4/b829194ed3dd11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/b3a07c86c4d811eba35c00155d207679_.jpg
CROPPED/RAW/Puma_4/b3022c9ed3dd11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/bd066bf6d3dd11ea91590242ac1c0002_.jpg
CROPPED/RAW/Puma_4/ead0e776c48411eb9d2b00155db4bfaf_.jpg
CROPPED/RAW/Puma_4/01638afcc48511eb9d2b00155db4bfaf_

In [ ]:
shutil.make_archive('Cropped_WildTrack', 'zip', rootdir)

In [ ]:
shutil.copy('Cropped_WildTrack.zip', PARENT_DIR + '/data/Cropped_WildTrack.zip')